In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"
out_folder = "/home/mlopez/EXEC/TimeMeansModels/"

In [10]:
shapefile = ("/home/mlopez/EXEC/Shapefiles/DDE_STF_20K_REG_FOR_VUE_S.shp")
poly = gpd.read_file(shapefile)
poly= poly.to_crs('epsg:4326')
poly

,OBJECTID,NO_REG_FOR,NM_REG_FOR,NO_UG_RAF,SUP_KM2,DATE_PUBLI,Shape_Leng,Shape_Area,D_GENERAL,geometry
0,1,01,BAS-SAINT-LAURENT,"011, 012",27645.7,2019-11-30,1.018112e+06,2.743725e+10,Sud-est,"POLYGON ((-68.24266 48.85438, -68.18955 48.875..."
1,2,02,SAGUENAY -LAC-SAINT-JEAN,"023, 024, 025, 027",102315.5,2019-11-30,2.642611e+06,1.011782e+11,Nord-est,"POLYGON ((-71.02818 52.21282, -71.02784 52.213..."
2,3,03,CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES,"035, 037",35573.9,2019-11-30,1.295862e+06,3.544054e+10,Centre du Québec,"POLYGON ((-70.24926 46.12110, -70.24936 46.120..."
3,4,04,MAURICIE-CENTRE-DU-QUÉBEC,"041, 042",52443.7,2019-11-30,1.903121e+06,5.216317e+10,Centre du Québec,"POLYGON ((-72.70529 45.75959, -72.70536 45.759..."
4,5,05,ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL,"051, 052",23252.3,2019-11-30,1.123268e+06,2.330905e+10,Secteur métropolitain et sud,"POLYGON ((-72.27288 45.61310, -72.27288 45.613..."
5,6,07,OUTAOUAIS,"071, 072, 073, 074",39030.0,2019-11-30,1.677902e+06,3.893718e+10,Sud-ouest,"POLYGON ((-74.72535 45.85802, -74.72646 45.852..."
6,7,08,ABITIBI-TEMISCAMINGUE,"081, 082, 083, 084, 085, 086",57124.2,2019-11-30,1.961835e+06,5.674050e+10,Nord-ouest,"POLYGON ((-77.86075 46.21053, -77.86104 46.210..."
7,8,09,COTE-NORD,"093, 094, 097",379435.8,2019-11-30,5.360494e+06,3.744732e+11,Nord-est,"POLYGON ((-70.02990 52.89020, -67.32805 52.884..."
8,9,10,NORD-DU-QUEBEC,"102, 105, 106, 107",0.1,2019-11-30,1.858192e+03,1.456492e+05,Nord-ouest,"POLYGON ((-78.76555 48.69057, -78.76556 48.690..."
9,10,10,NORD-DU-QUEBEC,"102, 105, 106, 107",48.3,2019-11-30,3.542201e+04,4.789565e+07,Nord-ouest,"POLYGON ((-78.65906 48.69034, -78.66015 48.690..."


# Select variable

In [4]:
filesSeas =  glob.glob(folder+"*rcp85_tg_mean_seasonal.nc")
files = glob.glob(folder+"*rcp85_tg_mean_annual.nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
print(files)
print(filesSeas)


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp85_tg_mean_annual.nc', '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp85_tg_mean_annual.nc', '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp85_tg_mean_annual.nc', '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp85_tg_mean_annual.nc', '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp85_tg_mean_annual.nc', '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp85_tg_mean_annual.nc', '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp85_tg_mean_annual.nc', '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp85_tg_mean_annual.nc', '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp85_tg_mean_annual.nc', '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp85_tg_mean_annual.nc', '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp85_tg_mean_annual.nc']
['/scen3/scenario/netcdf/ouranos/portrai

# Open Mask if already created or create it. 

In [7]:
dsEns= ens.create_ensemble(files)
dsEnsSeas = ens.create_ensemble(filesSeas)
dsEns


<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [11]:
if os.path.exists(shapefile.replace('.shp','.nc')):
    mask = xr.open_dataset(shapefile.replace('.shp','.nc'))
    mask = mask.__xarray_dataarray_variable__
    
else:
    mask= subset.create_mask(poly=poly, x_dim=dsEns.lon, y_dim=dsEns.lat)
    mask = mask.T
    mask.to_netcdf(shapefile.replace('.shp','.nc'))

# To create the Table

In [12]:
# Table 
# Similar to maps but with spatial avg as well

dsEns = dsEns.assign_coords({'region':mask})

# Mapping data - 30 year means
tmp1 = dsEns.sel(time=(dsEns.time.dt.year>=1981))
window = 30
time1 = tmp1.time[0::window]

# 30 y means
ds30yavgTable = tmp1.coarsen(time=window).mean()
ds30yavgTable['time'] = time1

# spatial mean
ds30yavgTable = ds30yavgTable.groupby(ds30yavgTable.region).mean()

perc30yavgTable = ens.ensemble_percentiles(ds30yavgTable)
perc30yavgTable 


<xarray.Dataset>
Dimensions:      (realization: 11, region: 13, time: 4)
Coordinates:
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
  * time         (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * region       (region) float64 0.0 1.0 2.0 3.0 4.0 ... 10.0 11.0 12.0 14.0
Data variables:
    tg_mean_p10  (region, time) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    tg_mean_p50  (region, time) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    tg_mean_p90  (region, time) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>

In [13]:
%time dfperT = perc30yavgTable.drop('realization').to_dataframe().dropna()


  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 1min 21s, sys: 5min 39s, total: 7min
Wall time: 20.2 s


In [14]:
dfperT.head()

tg_mean_p10  tg_mean_p50  tg_mean_p90
region time                                             
0.0    1981-01-01   275.938629   275.979034   276.177399
       2011-01-01   276.776093   277.254425   277.900269
       2041-01-01   278.462677   279.105682   280.052002
       2071-01-01   279.652985   281.866394   282.948700
1.0    1981-01-01   272.926788   273.026886   273.270935

In [15]:
dfReg1T = dfperT.query("region==0.0")
dfReg1T

tg_mean_p10  tg_mean_p50  tg_mean_p90
region time                                             
0.0    1981-01-01   275.938629   275.979034   276.177399
       2011-01-01   276.776093   277.254425   277.900269
       2041-01-01   278.462677   279.105682   280.052002
       2071-01-01   279.652985   281.866394   282.948700

# TODO
- This data (cell below) is only for one cell of the table, make for the other 2 periods ans 2 scenarios for every region

In [16]:
dfReg1T1980 = dfReg1T.query("time=='1981-01-01'")
dfReg1T1980

,,tg_mean_p10,tg_mean_p50,tg_mean_p90
region,time,,,
0.0,1981-01-01,275.938629,275.979034,276.177399


In [17]:
dfReg1T2041= dfReg1T.query("time=='2041-01-01'")
dfReg1T2041

,,tg_mean_p10,tg_mean_p50,tg_mean_p90
region,time,,,
0.0,2041-01-01,278.462677,279.105682,280.052002


In [18]:
dfReg1T2071 = dfReg1T.query("time=='2071-01-01'")
dfReg1T2071

,,tg_mean_p10,tg_mean_p50,tg_mean_p90
region,time,,,
0.0,2071-01-01,279.652985,281.866394,282.9487


In [19]:
col1 = ["Annuel","Percentiles (10 - 90)","Hiver","Percentiles (10 - 90)","Printemps","Percentiles (10 - 90)",
        "Ete","Percentiles (10 - 90)","Automne","Percentiles (10 - 90)"]

In [20]:
dfTable = pd.DataFrame()
dfTable["Saison"] = col1

In [21]:
dfTable

,Saison
0,Annuel
1,Percentiles (10 - 90)
2,Hiver
3,Percentiles (10 - 90)
4,Printemps
5,Percentiles (10 - 90)
6,Ete
7,Percentiles (10 - 90)
8,Automne
9,Percentiles (10 - 90)


In [22]:
dfReg1T1980.tg_mean_p50[0]

time
1981-01-01    275.979034
Name: tg_mean_p50, dtype: float64

In [23]:
dfTable["1981-2010"] = [dfReg1T1980.tg_mean_p50.values[0], (str(round(dfReg1T1980.tg_mean_p10.values[0],2))
                                                            +" - "+str(round(dfReg1T1980.tg_mean_p90.values[0],2))),str(1)+"-"+str(99),2,3,4,5,6,7,8]
dft2 = dfTable.set_index("Saison")

In [24]:
dft2.to_csv('dfTable.csv')